# Functional Programming in Scala Chapter 5

## Table of Contents
- [Stream Trait, Object, and Case Classes](#section0)
- [Exercise 5.1: toList](#section1)
- [Exercise 5.2: take and drop](#section2)
- [Exercise 5.3: takeWhile](#section3)
- [Exercise 5.4: forAll](#section4)
- [Exercise 5.5: foldRight and takeWhile](#section5)
- [Exercise 5.6: headOption](#section6)
- [Exercise 5.7: map, filter, append, and flatMap](#section7)
- [Exercise 5.8: infinite constant](#section8)
- [Exercise 5.9: infinite increase](#section9)
- [Exercise 5.10: fibonacci Infinite Sequence](#section10)
- [Exercise 5.11: unfold](#section11)
- [Exercise 5.12 : rewrite using unfold](#section12)
- [Exercise 5.13: rewrites using unfold round 2](#section13)
- [Exercise 5.14: startsWith](#section14)
- [Exercise 5.15: tails using unfold](#section15)
- [Exercise 5.16: scanRight](#section16)

<a id='section0'></a>

### Stream Trait, Object, and Case Classes

In [2]:
//here is the code provided by the author used in all the exercises below

sealed trait Stream[+A]//create stream trait
case object Empty extends Stream[Nothing]// generate empty Stream element
case class Cons[+A](h: () => A, t: () => Stream[A]) extends Stream[A]//generate nonempty Stream element

object Stream{
    def cons[A](hd: => A, tl: => Stream[A]):Stream[A] = {//lazy constructor of Stream
        lazy val head = hd
        lazy val tail = tl
        Cons(() => head, () => tail)
    }
    def empty[A]: Stream[A] = Empty//generate empty stream
    
    def apply[A](as: A*): Stream[A] = {//apply method for stream that calls the lazy constructor
        if(as.isEmpty) Empty
        else cons(as.head,apply(as.tail:_*))
    }
}

defined trait Stream
defined object Empty
defined class Cons
defined object Stream

<a id='section1'></a>

### Exercise 5.1: toList
Write a function to convert a Stream to a List, which will force its evaluation and let you look at it in the REPL. You can convert to the regular List type in the standard library. You can place this and other functions that operate on a Stream inside the Stream trait.

In [6]:
def toList[A](strm: Stream[A]): List[A] = {
    //use tail recursion to convert a stream to a list
    def go(iterStrm: Stream[A], acc: List[A]): List[A] ={//iterator that produces a list
        iterStrm match{//patteren matching for stream
            case Empty => acc//return list when iterate through all elements
            case Cons(h,t) => go(t(),acc++List(h()))//add head to list and iterate through the remaining elements of stream
        }
    }
    go(strm,List())
}

defined function toList

In [4]:
toList(Stream(1,2,3,4))

res2: List[Int] = List(1, 2, 3, 4)

<a id='section2'></a>

### Exercise 5.2: take and drop
Write the function take(n) for returning the first n elements of a Stream, and drop(n) for skipping the first n elements of a Stream.

In [4]:
def take[A](strm : Stream[A], n: Int): Stream[A] = {//way to take the first n elements from a stream
    def go(iterStrm: Stream[A], acc:Int): Stream[A]={//iterate through stream and take first n elements
        if(acc==n) Empty//if we have added the n elements then stop iteration
        else iterStrm match{//if have not reached n elements then pattern match
            case Empty => Empty//stream is empty stop iteration
            case Cons(h,t) => Stream.cons(h(),go(t(),acc+1))//else add the head to the stream and keep iterating
        }
    }
    go(strm,0)
}

defined function take

In [6]:
toList(take(Stream(1,2,3,4),2))

res4: List[Int] = List(1, 2)

In [7]:
def drop[A](strm: Stream[A], n: Int): Stream[A] = {//drop the first n elements from a stream
    def go(iterStrm: Stream[A], acc: Int): Stream[A] = {//iterate through stream
        if(acc==n) iterStrm//if we have dropped n elements then return remaining stream
        else iterStrm match{//haven't dropped n elements yet keep iterating
            case Empty => Empty//stream is empty then stop iteration
            case Cons(h,t) => go(t(),acc+1)//else drop the head and keep iterating
        }
    }
    go(strm,0)
}

defined function drop

In [8]:
toList(drop(a,2))

: 

<a id='section3'></a>

### Exercise 5.3: takeWhile
Write the function takeWhile for returning all starting elements of a Stream that match the given predicate.

In [9]:
def takeWhile[A](strm:Stream[A])(f: A => Boolean): Stream[A] ={//keep adding the first elements in a stream to a new stream
    //until f evaluates to false
    def go(iterStrm: Stream[A]): Stream[A] ={//iterate through stream
        iterStrm match{//pattern match on stream
            case Empty => Empty//empty stream then return empty stream
            case Cons(h,t) => {//non empty stream
                if(f(h())) Stream.cons(h(),go(t()))//head evaluates to true add to stream
                else Empty//otherwise stop iteration and return empty stream
            }
        }
    }
    go(strm)
}

defined function takeWhile

In [10]:
toList(takeWhile(Stream(1,2,3,4))(_%2==1))

res7: List[Int] = List(1)

<a id='section4'></a>

### Exercise 5.4: forAll
Implement forAll, which checks that all elements in the Stream match a given predicate. Your implementation should terminate the traversal as soon as it encounters a nonmatching value.

In [11]:
def forAll[A](strm:Stream[A])(f: A => Boolean): Boolean ={//see if f evaluates true for every element in a stream
    strm match{//pattern match on stream
        case Empty => true//empty element in a stream automatically evaluates to true
        case Cons(h,t) => f(h()) && forAll(t())(f)//apply f to the head apply the and operator and keep iterator
                                                  //as soon as hit a false the iteration will stop
    }
}

defined function forAll

In [12]:
forAll(Stream(2,4,1,8))({i => println(i); i%2==0})

2
4
1


res9: Boolean = false

<a id='section5'></a>

### Exercise 5.5: foldRight and takeWhile
Use foldRight to implement takeWhile.

In [3]:
def foldRight[A,B](strm: Stream[A], z: => B)(f:(A, => B) => B): B ={//recusively apply f to every element of a stream
    //and a base accumulator value(lazily)  though f will not return a value until it reaches the last element of a stream
    //and that value becomes the new accumulator value passed to the second to last element and this process continues until
    //we reach the head of the stream where finally the result of applying f to every element is returned
    strm match{//pattern match on stream
        case Empty => z//if stream is empty then return base value
        case Cons(h,t) => f(h(),foldRight(t(),z)(f))//else apply f to the head of the stream and the accumulative result 
        //of applying f to the remaining elements of the stream and the base value z
    }
}

defined function foldRight

In [14]:
def takeWhile[A](strm:Stream[A])(f:A => Boolean): Stream[A] ={//same takeWhile as above but uses foldRight
    foldRight(strm,Empty: Stream[A])((a, b) => if(f(a)) Stream.cons(a,b) else Empty)
}

defined function takeWhile

In [15]:
toList(takeWhile(Stream(1,2,3,4))(_%2==1))

res12: List[Int] = List(1)

In [16]:
def forAllfr[A](strm: Stream[A])(f: A => Boolean): Boolean = {//same forAll as above but uses fold right
    foldRight(strm,true)((a,b) => f(a)&&b)
}

defined function forAllfr

In [17]:
forAllfr(Stream(2,4,1,8))({i => println(i); i%2==0})

2
4
1


res14: Boolean = false

<a id='section6'></a>

###  Exercise 5.6: headOption
Hard: Implement headOption using foldRight.

In [18]:
def headOption[A](strm: Stream[A]): Option[A] ={//returns the head of a stream as an option using foldRight
    foldRight(strm, None:Option[A])((a,b) => a match{
        case a => Some(a)
        case _ => b
    })//base value is None, we look at the head and if it exists we wrap it in a option and return it otherwise return
    //the base value None
}

defined function headOption

In [19]:
headOption(Stream(1,2,3,4,5,6))
headOption(Empty)

res16_0: Option[Int] = Some(1)
res16_1: Option[Nothing] = None

<a id='section7'></a>

### Exercise 5.7: map, filter, append, and flatMap
Implement map, filter, append, and flatMap using foldRight. The append method should be non-strict in its argument.

In [20]:
def map[A,B](strm: Stream[A])(f:(=>A) => B):Stream[B] = {//lazily apply f to every element of a stream
    foldRight(strm,Empty:Stream[B])((a,b) => Stream.cons(f(a),b))//use foldRight, base value is an empty stream
    //for every element apply f and add it to the new stream
}

defined function map

In [21]:
def filter[A](strm: Stream[A])(f:(=>A) => Boolean):Stream[A] = {//lazily apply f to every element of stream and only
    //keep the elements that evaluate to true in the new stream 
    //uses foldRight
    foldRight(strm,Empty:Stream[A])((a,b) => if(f(a)) Stream.cons(a,b) else b)
}

defined function filter

In [22]:
def append[A](strm:Stream[A],strm2: Stream[A]): Stream[A] = {//take two streams and convert to one stream
    //uses foldRight where the base value is strm2
    foldRight(strm,strm2)((a,b) => Stream.cons(a,b))
}

defined function append

In [23]:
def flatMap[A,B](strm: Stream[A])(f:A =>Stream[B]):Stream[B] ={
    //same as map except f returns a stream of B as opposed to B
    //also append is used in the function passed to foldRight to concatenate all the streams outputed by f
    foldRight(strm,Empty:Stream[B])((a,b)=> append(f(a),b))
}

defined function flatMap

In [24]:
toList(append(Stream(1,2,3),Stream(4,5,6)))

res21: List[Int] = List(1, 2, 3, 4, 5, 6)

In [25]:
toList(filter(Stream(1,2,3))(_%2==0))

res22: List[Int] = List(2)

In [26]:
toList(flatMap(Stream(Stream(1,2,3),Stream(4,5,6)))(i => map(i)(_*2)))

res23: List[Int] = List(2, 4, 6, 8, 10, 12)

<a id='section8'></a>

### Exercise 5.8: infinite constant
Generalize ones slightly to the function constant, which returns an infinite Stream of a given value.

In [27]:
def constants[A](a: A):Stream[A] ={
    //generates an infinite stream of value passed
    //since cons is a lazy constructor none of the infinite values are generted until called upon
    Stream.cons(a,constants(a))
}

defined function constants

In [28]:
val aa = constants(2)

aa: Stream[Int] = Cons(<function0>,<function0>)

In [29]:
forAll(aa)(_%2==1)
takeWhile(aa)(_==2)
toList(take(aa,4))

res26_0: Boolean = false
res26_1: Stream[Int] = Cons(<function0>,<function0>)
res26_2: List[Int] = List(2, 2, 2, 2)

<a id='section9'></a>

### Exercise 5.9: infinite increase
Write a function that generates an infinite stream of integers, starting from n, then n + 1, n + 2, and so on.

In [30]:
def from(n:Int): Stream[Int] ={
    //generate infinite stream of integers starting with n and each proceeding value after the head is one greater
    //since cons is a lazy constructor none of the infinite values are generated until called upon
    Stream.cons(n,from(n+1))
}

defined function from

In [31]:
toList(take(from(4),10))

res28: List[Int] = List(
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13
)

<a id='section10'></a>

### Exercise 5.10: fibonacci Infinite Sequence
Write a function fibs that generates the infinite stream of Fibonacci numbers: 0, 1, 1, 2, 3, 5, 8, and so on.

In [32]:
def fibInf : Stream[Int] = {
    //generate infinte stream of fibonacci sequence
    def go(n: Int,n1: Int): Stream[Int] ={
        Stream.cons(n,go(n1,n+n1))
    }
    go(0,1)
}

defined function fibInf

In [33]:
toList(take(fibInf,10))

res30: List[Int] = List(
  0,
  1,
  1,
  2,
  3,
  5,
  8,
  13,
  21,
  34
)

<a id='section11'></a>

### Exercise 5.11: unfold
Write a more general stream-building function called unfold. It takes an initial state, and a function for producing both the next state and the next value in the generated stream.

In [34]:
def unfold[A,S](z:S)(f:S => Option[(A,S)]): Stream[A] = {//build a stream given an initial starting value and a function
    //to generate the next element
    f(z) match{//pattern match on the output of f
        case Some((h,t)) => Stream.cons(h, unfold(t)(f))//if the output is nonempty add the e output to the new stream
        //and keep iterating
        case None => Empty//hit an exception stop iterating return empty stream
    }
}

defined function unfold

<a id='section12'></a>

### Exercise 5.12 : rewrite using unfold
Write fibs, from, constant, and ones in terms of unfold.

In [35]:
val onesUF: Stream[Int] = unfold(1)(a => Some(1,1))//infinite stream of ones using unfold

onesUF: Stream[Int] = Cons(<function0>,<function0>)

In [36]:
toList(take(onesUF,5))

res33: List[Int] = List(1, 1, 1, 1, 1)

In [37]:
def constantUF[A](n: A): Stream[A] ={
    //same as constant above but uses unfold
    unfold(n)(a => Option(n,n))
}

defined function constantUF

In [38]:
toList(take(constantUF("a"),5))

res35: List[String] = List("a", "a", "a", "a", "a")

In [39]:
def fibsUF: Stream[Int] = {
    //same as fib above but uses unfold
    unfold((0,1))(a => Option(a._1,(a._2,a._1+a._2)))
}

defined function fibsUF

In [40]:
toList(take(fibsUF,10))

res37: List[Int] = List(
  0,
  1,
  1,
  2,
  3,
  5,
  8,
  13,
  21,
  34
)

<a id='section13'></a>

### Exercise 5.13: rewrites using unfold round 2
Use unfold to implement map, take, takeWhile, zipWith (as in chapter 3), and zipAll. The zipAll function should continue the traversal as long as either stream has more elements—it uses Option to indicate whether each stream has been exhausted.

In [41]:
def mapUF[A,B](st: Stream[A])(f: A => B): Stream[B] = {
    //rewrite of map using unfold
    unfold(st){a => //our function takes the stream
        a match{//then pattern matches on the stream
            case Cons(h,t) => Some(f(h()),t())//if the stream is nonempty apply f to the head and place in Some so it 
            //can be added to to the stream and also pass the tail to the stream so we can keep iterating
            case Empty => None//if the stream is empty return None so iteration will stop
        }
    }
}

defined function mapUF

In [42]:
toList(mapUF(Stream(1,2,3))(_*3))

res39: List[Int] = List(3, 6, 9)

In [43]:
def take[A](st:Stream[A], n: Int): Stream[A] ={
    //rewrite of take using unfold
    unfold((st,0)){a => //our base value is a tuple containing our stream and the initial 
                       //amount of elements in our new stream, 0
        a match{// pattern match on the tuple
            case (Cons(h,t),b) if b<n => Some(h(),(t(),b+1))// if our stream is nonempty and we have added less than n
            //elements to our stream then add the head to our stream, keep iterating over tail and increment up our counter
            case _ => None// if the list is empty or we have added our n elements to the new stream then stop iteration
        }
        
    }
}

defined function take

In [44]:
toList(take(Stream(1,2,3),2))

res41: List[Int] = List(1, 2)

In [45]:
def takeWhile[A](st: Stream[A])(f: A => Boolean): Stream[A] = {
    //rewrite of takeWhile using unfold
    unfold(st){a => //the initial stream is our initial value
              a match{//pattern match on a
                  case Cons(h,t) if f(h()) => Some(h(),t())//if the stream is nonempty and the head evaluates to true
                  //then add the head to the new stream and keep iterating over the tail
                  case _ => None// if the stream is empty or the head evaluated to false then stop iteration
              }}
}

defined function takeWhile

In [46]:
toList(takeWhile(Stream(1,1,2,3))(_%2==1))

res43: List[Int] = List(1, 1)

In [47]:
def zipWith[A,B,C](sta: Stream[A], stb: Stream[B])(f:(A,B) => C): Stream[C] = {
    //take two streams apply f to their pairwise elements to generate a new stream of the output of f
    //using unfold
    unfold((sta,stb)){a => //take the tuple of the two streams as initial value
                     a match{//pattern match on the tuple
                         case (Cons(ha,ta),Cons(hb,tb)) => Some(f(ha(),hb()),(ta(),tb()))//both nonempty streams
                         //then apply f to the two heads and keep iterating over the two tails
                         case _ => None// if the streams are empty stop iteration
                     }}
}

defined function zipWith

In [48]:
toList(zipWith(Stream(1,1,2,3),Stream(1,1,2,3))(_+_))

res45: List[Int] = List(2, 2, 4, 6)

In [49]:
def zipAll[A,B](sta:Stream[A],stb: Stream[B]): Stream[(Option[A],Option[B])] = {
    //takes two streams that could be of different length and produce a new stream of their pairwise elements and
    // if an element does not have a corresponding element in the the other stream a None is used as a placeholder
    //uses unfold
    unfold((sta,stb)){a => //tuple of the two streams is our initial value
                     a match{//pattern match on the tuple
                         case (Cons(ha,ta),Cons(hb,tb)) => Some((Some(ha()),Some(hb())),(ta(),tb()))//if both streams
                         //are nonempty wrap both heads in some and place them in a tuple to be added to our
                         //new stream and keep iterating over the tails
                         case (Cons(ha,ta),_) => Some((Some(ha()),None),(ta(),Empty))//if the second stream passed to 
                         //zipAll is empty wrap the first streams head in Some and place it in a tuple with None and
                         //keep iterating over the first stream's tail
                         case (_,Cons(hb,tb)) => Some((None,Some(hb())),(Empty,tb()))//if the first stream passed to 
                         //zipAll is empty wrap the second streams head in Some and place it in a tuple with None and
                         //keep iterating over the second stream's tail
                         case _ => None//if both streams are empty stop iteration
                     }}
}

defined function zipAll

In [50]:
toList(zipAll(Stream(1,2,3),Stream(1,2,3,4)))

res47: List[(Option[Int], Option[Int])] = List(
  (Some(1), Some(1)),
  (Some(2), Some(2)),
  (Some(3), Some(3)),
  (None, Some(4))
)

<a id='section14'></a>

### Exercise 5.14: startsWith
Hard: Implement startsWith using functions you’ve written. It should check if one Stream is a prefix of another. For instance, Stream(1,2,3) startsWith Stream(1,2) would be true.

In [51]:
def startsWith[A](sta: Stream[A],stb: Stream[A]): Boolean ={
    //take two streams and check if they they if one stream is equal to the beginnig of the other stream
    forAll(filter(zipAll(sta,stb))(j => j._1!=None && j._2!=None))(i => i._1==i._2)
    //first the two streams are zipAll'ed together because one stream could be shorter than the other
    //then we filter the output of zipAll to be the stream containing tuples where neither value is None this way 
    //we only check for equality with elements that are in the two passed streams
    //finally we check to make sure all the pairwise elements are equal to one another
    //all of these functions are lazy and as soon as we hit a false the function will terminate and return false
    //the only way all elements are evaluated is if all pairwise elements in the zipAll (that are not none)
    //are equal and the last pair could or could not be equal
    //ie the second test will iterate over all pairwise elements in the zipAll (that are not none)
}

defined function startsWith

In [52]:
startsWith(Stream(1,2,3,4),Stream(1,2))
startsWith(Stream(1,2,3,4),Stream(1,2,5))

res49_0: Boolean = true
res49_1: Boolean = false

<a id='section15'></a>

### Exercise 5.15: tails using unfold
Implement tails using unfold. For a given Stream, tails returns the Stream of suffixes of the input sequence, starting with the original Stream. For example, given Stream(1,2,3), it would return Stream(Stream(1,2,3), Stream(2,3), Stream(3), Stream()).

In [73]:
def tails[A](st: Stream[A]): Stream[Stream[A]] ={
    //take a stream and return a stream of streams where the nth element of the Stream is a Stream containing the nth
    //element of the stream and all of its proceeding elements
    //uses unfold
    unfold(st){a => //stream passed if out initial value
              a match{ //pattern match on the stream
                  case Cons(h,t) => Some(Cons(h,t),t())//if the stream is nonempty add the stream to our new stream of 
                  //streams and keep iterating over the tail
                  case Empty => None//if the list is empty then stop iteration
              }}
}

defined function tails

In [74]:
toList(mapUF(tails(Stream(1,2,3,4)))(toList))

res66: List[List[Int]] = List(
  List(1, 2, 3, 4),
  List(2, 3, 4),
  List(3, 4),
  List(4)
)

<a id='section16'></a>

### Exercise 5.16: scanRight
Hard: Generalize tails to the function scanRight, which is like a foldRight that returns a stream of the intermediate results. For example:

In [4]:
def scanRight[A,B](st:Stream[A], z: B)(f: (A, =>B) => B): Stream[B] = {
    //implementation of scanRight in linear time using foldRight
    //Stream is taken f is applied to nth element and all its proceeding elements and the accumalated result 
    //becomes the nth element in the new stream, for every element in the passed stream 
    foldRight(st,(z,Stream(z))){(a,b) => //foldRight on the passed stream with base value of tuple with 
                                //base value passed and that base value in a stream
                               lazy val c = b //lazily evaluate the accumulator value
                               val out = f(a,c._1) //apply f to the head and accumulated value lazily
                               (out,Stream.cons(out,c._2))}._2 //add the the output to the new stream and keep iterating
}

defined function scanRight

In [7]:
toList(scanRight(Stream(1,2,3),0)(_+_))

res4: List[Int] = List(6, 5, 3, 0)

Unfold can be used in this problem.  It is suboptimal to the result above but it can be done, as shown below.

In [79]:
def reverse[A](st: Stream[A]): Stream[A] = {
    //reverse elements of stream
    st match{//pattern match stream
        case Cons(h,t) => append(reverse(t()),Stream(h()))//add head to the end of the stream and keep iterating over tail
        case _ => Empty//when stream is empty stop iterating
    }
}

defined function reverse

In [95]:
def scanRight[A,B](st:Stream[A],z: => B)(f: (A,=>B) => B): Stream[B] ={
    //scan right using unfold
    val rev = reverse(st)//reverse stream
    val revOut = unfold((rev,z)){a => //initial value of our reversed stream and base value
              a match{ //pattern match on our tuple
                  case (Cons(h,t),l) => {Some(f(h(),l),(t(),zz))}//apply f to our head and accumulated value and keep iterating
                  case _ => None//once stream is empty stop iteration
              }}
    append(reverse(revOut),Stream(z))//reverse the result of our unfold and add the base value to the end of the list
}

defined function scanRight

In [94]:
toList(scanRight(Stream(1,2,3),0)(_+_))

res78: List[Int] = List(6, 5, 3, 0)